In [1]:
# Add execute permissions to binaries
!chmod +x dssp
!chmod +x DAlphaBall.gcc

# Verify permissions
!ls -l dssp
!ls -l DAlphaBall.gcc

-rwxr-xr-x@ 1 satishgaurav  staff  877904 Jan 28 10:21 dssp
-rwxr-xr-x@ 1 satishgaurav  staff  345824 Jan 28 10:21 DAlphaBall.gcc


In [2]:
####################################
################ BioPython functions
####################################
### Import dependencies
import os
import math
import numpy as np
from collections import defaultdict
from scipy.spatial import cKDTree
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.Polypeptide import is_aa

# analyze sequence composition of design
def validate_design_sequence(sequence, num_clashes, advanced_settings):
    note_array = []

    # Check if protein contains clashes after relaxation
    if num_clashes > 0:
        note_array.append('Relaxed structure contains clashes.')

    # Check if the sequence contains disallowed amino acids
    if advanced_settings["omit_AAs"]:
        restricted_AAs = advanced_settings["omit_AAs"].split(',')
        for restricted_AA in restricted_AAs:
            if restricted_AA in sequence:
                note_array.append('Contains: '+restricted_AA+'!')

    # Analyze the protein
    analysis = ProteinAnalysis(sequence)

    # Calculate the reduced extinction coefficient per 1% solution
    extinction_coefficient_reduced = analysis.molar_extinction_coefficient()[0]
    molecular_weight = round(analysis.molecular_weight() / 1000, 2)
    extinction_coefficient_reduced_1 = round(extinction_coefficient_reduced / molecular_weight * 0.01, 2)

    # Check if the absorption is high enough
    if extinction_coefficient_reduced_1 <= 2:
        note_array.append(f'Absorption value is {extinction_coefficient_reduced_1}, consider adding tryptophane to design.')

    # Join the notes into a single string
    notes = ' '.join(note_array)

    return notes

# temporary function, calculate RMSD of input PDB and trajectory target
def target_pdb_rmsd(trajectory_pdb, starting_pdb, chain_ids_string):
    # Parse the PDB files
    parser = PDBParser(QUIET=True)
    structure_trajectory = parser.get_structure('trajectory', trajectory_pdb)
    structure_starting = parser.get_structure('starting', starting_pdb)
    
    # Extract chain A from trajectory_pdb
    chain_trajectory = structure_trajectory[0]['A']
    
    # Extract the specified chains from starting_pdb
    chain_ids = chain_ids_string.split(',')
    residues_starting = []
    for chain_id in chain_ids:
        chain_id = chain_id.strip()
        chain = structure_starting[0][chain_id]
        for residue in chain:
            if is_aa(residue, standard=True):
                residues_starting.append(residue)
    
    # Extract residues from chain A in trajectory_pdb
    residues_trajectory = [residue for residue in chain_trajectory if is_aa(residue, standard=True)]
    
    # Ensure that both structures have the same number of residues
    min_length = min(len(residues_starting), len(residues_trajectory))
    residues_starting = residues_starting[:min_length]
    residues_trajectory = residues_trajectory[:min_length]
    
    # Collect CA atoms from the two sets of residues
    atoms_starting = [residue['CA'] for residue in residues_starting if 'CA' in residue]
    atoms_trajectory = [residue['CA'] for residue in residues_trajectory if 'CA' in residue]
    
    # Calculate RMSD using structural alignment
    sup = Superimposer()
    sup.set_atoms(atoms_starting, atoms_trajectory)
    rmsd = sup.rms
    
    return round(rmsd, 2)

# detect C alpha clashes for deformed trajectories
def calculate_clash_score(pdb_file, threshold=2.4, only_ca=False):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    atoms = []
    atom_info = []  # Detailed atom info for debugging and processing

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    if only_ca and atom.get_name() != 'CA':
                        continue
                    atoms.append(atom.coord)
                    atom_info.append((chain.id, residue.id[1], atom.get_name(), atom.coord))

    tree = cKDTree(atoms)
    pairs = tree.query_pairs(threshold)

    valid_pairs = set()
    for (i, j) in pairs:
        chain_i, res_i, name_i, coord_i = atom_info[i]
        chain_j, res_j, name_j, coord_j = atom_info[j]

        # Exclude clashes within the same residue
        if chain_i == chain_j and res_i == res_j:
            continue

        # Exclude directly sequential residues in the same chain for all atoms
        if chain_i == chain_j and abs(res_i - res_j) == 1:
            continue

        # If calculating sidechain clashes, only consider clashes between different chains
        if not only_ca and chain_i == chain_j:
            continue

        valid_pairs.add((i, j))

    return len(valid_pairs)

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

# identify interacting residues at the binder interface
def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues

# calculate secondary structure percentage of design
def calc_ss_percentage(pdb_file, advanced_settings, chain_id="B", atom_distance_cutoff=4.0):
    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]  # Consider only the first model in the structure

    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')

    # Prepare to count residues
    ss_counts = defaultdict(int)
    ss_interface_counts = defaultdict(int)
    plddts_interface = []
    plddts_ss = []

    # Get chain and interacting residues once
    chain = model[chain_id]
    interacting_residues = set(hotspot_residues(pdb_file, chain_id, atom_distance_cutoff).keys())

    for residue in chain:
        residue_id = residue.id[1]
        if (chain_id, residue_id) in dssp:
            ss = dssp[(chain_id, residue_id)][2]  # Get the secondary structure
            ss_type = 'loop'
            if ss in ['H', 'G', 'I']:
                ss_type = 'helix'
            elif ss == 'E':
                ss_type = 'sheet'

            ss_counts[ss_type] += 1

            if ss_type != 'loop':
                # calculate secondary structure normalised pLDDT
                avg_plddt_ss = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_ss.append(avg_plddt_ss)

            if residue_id in interacting_residues:
                ss_interface_counts[ss_type] += 1

                # calculate interface pLDDT
                avg_plddt_residue = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_interface.append(avg_plddt_residue)

    # Calculate percentages
    total_residues = sum(ss_counts.values())
    total_interface_residues = sum(ss_interface_counts.values())

    percentages = calculate_percentages(total_residues, ss_counts['helix'], ss_counts['sheet'])
    interface_percentages = calculate_percentages(total_interface_residues, ss_interface_counts['helix'], ss_interface_counts['sheet'])

    i_plddt = round(sum(plddts_interface) / len(plddts_interface) / 100, 2) if plddts_interface else 0
    ss_plddt = round(sum(plddts_ss) / len(plddts_ss) / 100, 2) if plddts_ss else 0

    return (*percentages, *interface_percentages, i_plddt, ss_plddt)

def calculate_percentages(total, helix, sheet):
    helix_percentage = round((helix / total) * 100,2) if total > 0 else 0
    sheet_percentage = round((sheet / total) * 100,2) if total > 0 else 0
    loop_percentage = round(((total - helix - sheet) / total) * 100,2) if total > 0 else 0

    return helix_percentage, sheet_percentage, loop_percentage

In [3]:
advanced_settings = {
    "dssp_path": "/usr/local/bin/mkdssp",
    "omit_AAs": "C",
}
pdb_file = "./../target/1yi5.pdb"
calc_ss_percentage(pdb_file=pdb_file, advanced_settings=advanced_settings)

(8.33, 44.12, 47.55, 15.79, 52.63, 31.58, 0.39, 0.39)

In [4]:
import pandas as pd

AA_PROPERTIES = {
        "ALA": "nonpolar",
        "ARG": "positive",
        "ASN": "polar",
        "ASP": "negative",
        "CYS": "polar",
        "GLN": "polar",
        "GLU": "negative",
        "GLY": "nonpolar",
        "HIS": "positive",
        "ILE": "nonpolar",
        "LEU": "nonpolar",
        "LYS": "positive",
        "MET": "nonpolar",
        "PHE": "nonpolar",
        "PRO": "nonpolar",
        "SER": "polar",
        "THR": "polar",
        "TRP": "nonpolar",
        "TYR": "polar",
        "VAL": "nonpolar",
        "SEC": "polar"
    }

def create_structure_df(pdb_file, advanced_settings, chain_id=None, atom_distance_cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    
    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Prepare data for DataFrame
    data = []
    
    # Process either specific chain or all chains
    chains = [model[chain_id]] if chain_id else model.get_chains()
    
    for chain in chains:
        current_chain_id = chain.id
        # Get interacting residues if chain_id is specified
        interacting_residues = set()
        if current_chain_id:
            interacting_residues = set(hotspot_residues(pdb_file, current_chain_id, atom_distance_cutoff).keys())
        
        for residue in chain:
            residue_id = residue.id[1]
            res_name = residue.get_resname()
            
            if (current_chain_id, residue_id) in dssp:
                dssp_data = dssp[(current_chain_id, residue_id)]
                ss = dssp_data[2]
                acc = dssp_data[3]
                phi = dssp_data[4]
                psi = dssp_data[5]
                
                ss_type = 'loop'
                if ss in ['H', 'G', 'I']:
                    ss_type = 'helix'
                elif ss == 'E':
                    ss_type = 'sheet'
                    
                avg_plddt = sum(atom.bfactor for atom in residue) / len(residue) / 100
                
                data.append({
                    'chain_id': current_chain_id,
                    'residue_id': residue_id,
                    'residue_name': res_name,
                    'property': AA_PROPERTIES.get(res_name, 'unknown'),
                    'ss_type': ss_type,
                    'ss_code': ss,
                    'accessibility': acc,
                    'phi': phi,
                    'psi': psi,
                    'plddt': round(avg_plddt, 2),
                    'is_interface': residue_id in interacting_residues if chain_id else None
                })
    
    return pd.DataFrame(data)


# Example usage:
pdb_file = "./../target/1yi5.pdb"
df = create_structure_df(pdb_file, advanced_settings)
df.head()

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,LEU,nonpolar,loop,-,0.719512,360.0,127.0,0.39,None
1,A,2,ASP,negative,loop,-,0.282209,-129.9,161.8,0.39,None
2,A,3,ARG,positive,helix,H,0.084677,-37.6,-52.7,0.39,None
3,A,4,ALA,nonpolar,helix,H,0.132075,-54.6,-52.6,0.39,None
4,A,5,ASP,negative,helix,H,0.441718,-63.2,-22.2,0.39,None


In [5]:
df.tail(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1346,J,59,THR,polar,loop,S,0.669014,-85.5,132.6,0.39,None
1347,J,60,ASP,negative,loop,S,0.680982,-46.9,108.6,0.39,None
1348,J,61,ASN,polar,loop,S,0.337580,60.3,80.0,0.39,None
1349,J,62,CYS,polar,loop,-,0.207407,-121.2,1.8,0.39,None
1350,J,63,ASN,polar,loop,-,0.019108,-127.0,30.2,0.39,None
1351,J,64,PRO,nonpolar,loop,-,0.470588,-64.0,-176.1,0.39,None
1352,J,65,PHE,nonpolar,loop,-,0.375635,-69.0,132.3,0.39,None
1353,J,66,PRO,nonpolar,loop,-,0.419118,-85.9,-179.3,0.40,None
1354,J,67,THR,polar,loop,-,0.894366,-139.4,76.7,0.41,None
1355,J,68,ARG,positive,loop,-,0.576613,-158.9,360.0,0.42,None


In [6]:
df.head(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,LEU,nonpolar,loop,-,0.719512,360.0,127.0,0.39,None
1,A,2,ASP,negative,loop,-,0.282209,-129.9,161.8,0.39,None
2,A,3,ARG,positive,helix,H,0.084677,-37.6,-52.7,0.39,None
3,A,4,ALA,nonpolar,helix,H,0.132075,-54.6,-52.6,0.39,None
4,A,5,ASP,negative,helix,H,0.441718,-63.2,-22.2,0.39,None
5,A,6,ILE,nonpolar,helix,H,0.011834,-76.3,-44.9,0.39,None
6,A,7,LEU,nonpolar,helix,H,0.085366,-65.9,-27.1,0.39,None
7,A,8,TYR,polar,helix,H,0.563063,-67.9,-40.0,0.40,None
8,A,9,ASN,polar,helix,H,0.515924,-55.9,-62.9,0.39,None
9,A,10,ILE,nonpolar,helix,H,0.017751,-64.9,-50.8,0.39,None


In [7]:
df_target = df[df['chain_id'] == 'F']

In [8]:
df_target.to_csv('1yi5_chain_F_secondary_structure_information.csv', index=False)

In [9]:
df_target[df_target['property'] == 'nonpolar']

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
202,F,1,ILE,nonpolar,loop,-,0.165680,360.0,135.5,0.39,None
205,F,4,PHE,nonpolar,sheet,E,0.258883,-69.4,112.2,0.39,None
206,F,5,ILE,nonpolar,loop,-,0.319527,-95.9,150.5,0.39,None
208,F,7,PRO,nonpolar,loop,S,0.294118,-93.0,23.6,0.39,None
210,F,9,ILE,nonpolar,loop,S,0.650888,-79.6,-14.2,0.40,None
216,F,15,PRO,nonpolar,loop,S,0.794118,-59.1,117.1,0.40,None
218,F,17,GLY,nonpolar,loop,-,0.142857,-83.0,112.5,0.40,None
220,F,19,VAL,nonpolar,sheet,E,0.126761,-137.0,141.5,0.39,None
226,F,25,TRP,nonpolar,sheet,E,0.149780,-147.4,149.1,0.39,None
229,F,28,ALA,nonpolar,loop,S,0.028302,-85.5,-40.6,0.39,None


In [10]:
df_target[df_target['property'] == 'nonpolar'].sort_values('accessibility', ascending=False)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
216,F,15,PRO,nonpolar,loop,S,0.794118,-59.1,117.1,0.40,None
210,F,9,ILE,nonpolar,loop,S,0.650888,-79.6,-14.2,0.40,None
249,F,48,VAL,nonpolar,loop,-,0.619718,-112.8,115.6,0.40,None
252,F,51,GLY,nonpolar,loop,S,0.619048,98.2,-37.0,0.39,None
244,F,43,ALA,nonpolar,loop,S,0.603774,-63.4,-49.8,0.39,None
265,F,64,PRO,nonpolar,loop,-,0.492647,-64.6,-177.9,0.39,None
267,F,66,PRO,nonpolar,loop,-,0.433824,-78.0,176.3,0.40,None
266,F,65,PHE,nonpolar,loop,-,0.365482,-73.0,133.8,0.40,None
247,F,46,PRO,nonpolar,loop,P,0.330882,-74.9,168.4,0.39,None
206,F,5,ILE,nonpolar,loop,-,0.319527,-95.9,150.5,0.39,None


In [11]:
df_target.sort_values('accessibility', ascending=False)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
251,F,50,THR,polar,loop,S,1.000000,-75.0,122.2,0.40,None
248,F,47,THR,polar,loop,P,0.978873,-69.0,154.8,0.39,None
268,F,67,THR,polar,loop,-,0.901408,-107.5,-22.5,0.40,None
216,F,15,PRO,nonpolar,loop,S,0.794118,-59.1,117.1,0.40,None
250,F,49,LYS,positive,loop,-,0.756098,-72.8,153.3,0.40,None
...,...,...,...,...,...,...,...,...,...,...,...
264,F,63,ASN,polar,loop,-,0.038217,-126.8,31.0,0.39,None
229,F,28,ALA,nonpolar,loop,S,0.028302,-85.5,-40.6,0.39,None
238,F,37,VAL,nonpolar,sheet,E,0.021127,-134.8,121.2,0.39,None
221,F,20,CYS,polar,sheet,E,0.000000,-95.5,130.2,0.39,None


In [13]:
from IPython.display import display, Markdown
display(Markdown(df_target.to_markdown()))

|     | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|----:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 202 | F          |            1 | ILE            | nonpolar   | loop      | -         |       0.16568   |  360   |  135.5 |    0.39 |                |
| 203 | F          |            2 | ARG            | positive   | sheet     | E         |       0.41129   |  -77.2 |  117.5 |    0.39 |                |
| 204 | F          |            3 | CYS            | polar      | sheet     | E         |       0         | -126.1 |  150.2 |    0.39 |                |
| 205 | F          |            4 | PHE            | nonpolar   | sheet     | E         |       0.258883  |  -69.4 |  112.2 |    0.39 |                |
| 206 | F          |            5 | ILE            | nonpolar   | loop      | -         |       0.319527  |  -95.9 |  150.5 |    0.39 |                |
| 207 | F          |            6 | THR            | polar      | loop      | S         |       0.140845  | -112.8 |  147.8 |    0.39 |                |
| 208 | F          |            7 | PRO            | nonpolar   | loop      | S         |       0.294118  |  -93   |   23.6 |    0.39 |                |
| 209 | F          |            8 | ASP            | negative   | loop      | S         |       0.466258  |  -89.2 |  152.2 |    0.4  |                |
| 210 | F          |            9 | ILE            | nonpolar   | loop      | S         |       0.650888  |  -79.6 |  -14.2 |    0.4  |                |
| 211 | F          |           10 | THR            | polar      | loop      | S         |       0.43662   | -101.8 |  165.2 |    0.39 |                |
| 212 | F          |           11 | SER            | polar      | sheet     | E         |       0.292308  |  -94   |  143.7 |    0.39 |                |
| 213 | F          |           12 | LYS            | positive   | sheet     | E         |       0.458537  | -140.3 |  161.5 |    0.4  |                |
| 214 | F          |           13 | ASP            | negative   | sheet     | E         |       0.656442  |  -90.2 |  102.3 |    0.4  |                |
| 215 | F          |           14 | CYS            | polar      | loop      | -         |       0.118519  | -106.6 |  100   |    0.4  |                |
| 216 | F          |           15 | PRO            | nonpolar   | loop      | S         |       0.794118  |  -59.1 |  117.1 |    0.4  |                |
| 217 | F          |           16 | ASN            | polar      | loop      | S         |       0.675159  |    8.7 |   70.7 |    0.39 |                |
| 218 | F          |           17 | GLY            | nonpolar   | loop      | -         |       0.142857  |  -83   |  112.5 |    0.4  |                |
| 219 | F          |           18 | HIS            | positive   | loop      | -         |       0.63587   |  -70.6 |   -3.3 |    0.4  |                |
| 220 | F          |           19 | VAL            | nonpolar   | sheet     | E         |       0.126761  | -137   |  141.5 |    0.39 |                |
| 221 | F          |           20 | CYS            | polar      | sheet     | E         |       0         |  -95.5 |  130.2 |    0.39 |                |
| 222 | F          |           21 | TYR            | polar      | sheet     | E         |       0.193694  | -124.1 |  153.5 |    0.39 |                |
| 223 | F          |           22 | THR            | polar      | sheet     | E         |       0.140845  | -135.3 |   84.4 |    0.39 |                |
| 224 | F          |           23 | LYS            | positive   | sheet     | E         |       0.321951  |  -69.2 |  138.9 |    0.4  |                |
| 225 | F          |           24 | THR            | polar      | sheet     | E         |       0.190141  | -149.2 |  124.5 |    0.4  |                |
| 226 | F          |           25 | TRP            | nonpolar   | sheet     | E         |       0.14978   | -147.4 |  149.1 |    0.39 |                |
| 227 | F          |           26 | CYS            | polar      | loop      | -         |       0.318519  |  -93   |  119.6 |    0.4  |                |
| 228 | F          |           27 | ASP            | negative   | loop      | -         |       0.0858896 | -101.9 | -149.4 |    0.39 |                |
| 229 | F          |           28 | ALA            | nonpolar   | loop      | S         |       0.0283019 |  -85.5 |  -40.6 |    0.39 |                |
| 230 | F          |           29 | PHE            | nonpolar   | loop      | T         |       0.0507614 |  -69.6 |   23.4 |    0.39 |                |
| 231 | F          |           30 | CYS            | polar      | helix     | H         |       0.0444444 | -107.2 |  -36.9 |    0.39 |                |
| 232 | F          |           31 | SER            | polar      | helix     | H         |       0.415385  |  -65.1 |  -25.5 |    0.39 |                |
| 233 | F          |           32 | ILE            | nonpolar   | helix     | H         |       0.224852  |  -84.1 |  -73   |    0.39 |                |
| 234 | F          |           33 | ARG            | positive   | helix     | H         |       0.0483871 |  -95.1 |   -2.2 |    0.39 |                |
| 235 | F          |           34 | GLY            | nonpolar   | loop      | -         |       0.214286  |  114.6 | -177.9 |    0.39 |                |
| 236 | F          |           35 | LYS            | positive   | loop      | -         |       0.326829  |  -58.7 |  146.8 |    0.39 |                |
| 237 | F          |           36 | ARG            | positive   | sheet     | E         |       0.314516  |  -73.8 |  146.9 |    0.39 |                |
| 238 | F          |           37 | VAL            | nonpolar   | sheet     | E         |       0.0211268 | -134.8 |  121.2 |    0.39 |                |
| 239 | F          |           38 | ASP            | negative   | sheet     | E         |       0.319018  | -106   |  114.1 |    0.39 |                |
| 240 | F          |           39 | LEU            | nonpolar   | sheet     | E         |       0.054878  |  -90   |  166.5 |    0.39 |                |
| 241 | F          |           40 | GLY            | nonpolar   | sheet     | E         |       0.047619  |  162.2 | -171.9 |    0.39 |                |
| 242 | F          |           41 | CYS            | polar      | sheet     | E         |       0.237037  | -122.4 |  162.8 |    0.4  |                |
| 243 | F          |           42 | ALA            | nonpolar   | sheet     | E         |       0.254717  | -163.5 |  168.6 |    0.39 |                |
| 244 | F          |           43 | ALA            | nonpolar   | loop      | S         |       0.603774  |  -63.4 |  -49.8 |    0.39 |                |
| 245 | F          |           44 | THR            | polar      | loop      | S         |       0.753521  | -138.3 |  169.9 |    0.4  |                |
| 246 | F          |           45 | CYS            | polar      | loop      | P         |       0.503704  |  -65.3 |  135.3 |    0.39 |                |
| 247 | F          |           46 | PRO            | nonpolar   | loop      | P         |       0.330882  |  -74.9 |  168.4 |    0.39 |                |
| 248 | F          |           47 | THR            | polar      | loop      | P         |       0.978873  |  -69   |  154.8 |    0.39 |                |
| 249 | F          |           48 | VAL            | nonpolar   | loop      | -         |       0.619718  | -112.8 |  115.6 |    0.4  |                |
| 250 | F          |           49 | LYS            | positive   | loop      | -         |       0.756098  |  -72.8 |  153.3 |    0.4  |                |
| 251 | F          |           50 | THR            | polar      | loop      | S         |       1         |  -75   |  122.2 |    0.4  |                |
| 252 | F          |           51 | GLY            | nonpolar   | loop      | S         |       0.619048  |   98.2 |  -37   |    0.39 |                |
| 253 | F          |           52 | VAL            | nonpolar   | loop      | -         |       0.197183  |  -51.9 |  143.8 |    0.39 |                |
| 254 | F          |           53 | ASP            | negative   | loop      | B         |       0.521472  | -123.2 |  126.6 |    0.4  |                |
| 255 | F          |           54 | ILE            | nonpolar   | loop      | -         |       0.284024  | -118   |  161.8 |    0.4  |                |
| 256 | F          |           55 | GLN            | polar      | loop      | -         |       0.717172  | -157.3 |  103.9 |    0.39 |                |
| 257 | F          |           56 | CYS            | polar      | sheet     | E         |       0.325926  |  -93.7 |  162.2 |    0.39 |                |
| 258 | F          |           57 | CYS            | polar      | sheet     | E         |       0.244444  | -173.1 |  177.3 |    0.39 |                |
| 259 | F          |           58 | SER            | polar      | loop      | -         |       0.715385  | -141.9 |   12.9 |    0.39 |                |
| 260 | F          |           59 | THR            | polar      | loop      | S         |       0.683099  |  -88.4 |  133.8 |    0.39 |                |
| 261 | F          |           60 | ASP            | negative   | loop      | S         |       0.662577  |  -50   |  107.9 |    0.4  |                |
| 262 | F          |           61 | ASN            | polar      | loop      | S         |       0.33121   |   63.7 |   81   |    0.39 |                |
| 263 | F          |           62 | CYS            | polar      | loop      | -         |       0.281481  | -125.6 |    6.3 |    0.39 |                |
| 264 | F          |           63 | ASN            | polar      | loop      | -         |       0.0382166 | -126.8 |   31   |    0.39 |                |
| 265 | F          |           64 | PRO            | nonpolar   | loop      | -         |       0.492647  |  -64.6 | -177.9 |    0.39 |                |
| 266 | F          |           65 | PHE            | nonpolar   | loop      | -         |       0.365482  |  -73   |  133.8 |    0.4  |                |
| 267 | F          |           66 | PRO            | nonpolar   | loop      | -         |       0.433824  |  -78   |  176.3 |    0.4  |                |
| 268 | F          |           67 | THR            | polar      | loop      | -         |       0.901408  | -107.5 |  -22.5 |    0.4  |                |
| 269 | F          |           68 | ARG            | positive   | loop      | -         |       0.516129  | -178.2 |  360   |    0.4  |                |